In [1]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

In [6]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [7]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [8]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [9]:
from sentence_transformers import SentenceTransformer

In [10]:
encoder = SentenceTransformer("all-mpnet-base-v2")

# Convert text column into a list
texts = df["text"].tolist()

# Encode all rows
vectors = encoder.encode(texts)

print(vectors.shape)   # (num_rows, 768)


(8, 768)


In [11]:
vectors.shape

(8, 768)

In [12]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [13]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [14]:
index.add(vectors)

In [15]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000025B7053B1B0> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [16]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [17]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [18]:
# faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [20]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load model
encoder = SentenceTransformer("all-mpnet-base-v2")

# Example dataset
texts = [
    "Tata Tiago iCNG price starts from ₹6.55 Lakh.",
    "Tata Punch iCNG features include dual-cylinder technology and high mileage.",
    "Maruti WagonR CNG offers good mileage and is a popular family car."
]

# Encode texts
vectors = encoder.encode(texts)

# Build FAISS index
d = vectors.shape[1]   # dimension
index = faiss.IndexFlatL2(d)
index.add(np.array(vectors))

# 🔹 Encode query
query = "What is the price of Tiago iCNG?"
new_vec = encoder.encode([query])  # Must be a 2D array

# 🔹 Search top 2 results
distances, I = index.search(new_vec, k=2)

print("Distances:", distances)
print("Indices:", I)
print("Matched Texts:", [texts[i] for i in I[0]])


Distances: [[0.4570098 1.2086828]]
Indices: [[0 1]]
Matched Texts: ['Tata Tiago iCNG price starts from ₹6.55 Lakh.', 'Tata Punch iCNG features include dual-cylinder technology and high mileage.']


In [21]:
I

array([[0, 1]])

In [22]:
I.tolist()

[[0, 1]]

In [23]:
row_indices = I.tolist()[0]
row_indices

[0, 1]

In [24]:
df.loc[row_indices]

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health


In [25]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query